## Alaska Berry Futures
### Wind speed and pollinators

February 16, 2023    
Using SNAP wind models to investigate how wind speed my affect pollinator movements in AK's future    

In [1]:
import requests
import csv
import requests
import pandas as pd
import os
from io import StringIO

In [2]:
working_directory = os.getcwd()
weather_stations =  pd.read_csv(working_directory + "\\SNAPwind_location_lookup_table-1.csv")

# Group the dataframe by the 'region' column, and get the unique values of 'icao_code' for each group
# save the codes as lowercase so it matches the csv names on the SNAP website
icao_codes_by_region = weather_stations.groupby('region')['icao_code'].apply(lambda x: x.str.lower().unique())

# Print the resulting dictionary of icao_codes by region
print(icao_codes_by_region)


region
Alaska Range Transition              [paaq, paed, paen, paho, pail, pamr, panc, pas...
Aleutian Meadows                      [pacd, padk, padq, padu, paph, pasd, pasn, pasy]
Bering Taiga                                [pabe, pacz, padl, paeh, pakn, pasm, paun]
Bering Tundra                                                 [paom, paot, pash, patc]
Coastal Rainforests                  [pacv, pael, pags, pagy, pajn, pakt, pamd, pan...
Copper River & Wrangell Mountains                                   [padt, pagk, pamx]
Intermontane Boreal                  [pabi, pabt, paei, pafa, paga, paim, pamc, pam...
North Slope                                             [paba, pabr, pafb, palu, pasc]
Name: icao_code, dtype: object


In [3]:
def download_csv(url):
    """
    Downloads a CSV file from a URL and returns its contents as a Pandas DataFrame.

    :param url: The URL of the CSV file to download.
    :return: A Pandas DataFrame containing the contents of the CSV file.
    """
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    # Print an error message if the request failed
    if response.status_code != 200:
        raise ValueError(f"Failed to download CSV from {url}. Status code: {response.status_code}")
        
    # Load the content of the response into a Pandas DataFrame
    content = response.content.decode('utf-8')
    data = pd.read_csv(StringIO(content))
    
    # Return the parsed data
    return data


In [4]:
def download_csvs_for_locations(regions, model):
    """
    Download CSV data for multiple regions.

    Args:
        regions (list): A list of region names to download CSV data for.

    Returns:
        dict: A dictionary containing a Pandas DataFrame for each location.
              The location name is used as the key in the dictionary.
    """
    # URL format string
    url_format = {
        'past_asos': 'http://data.snap.uaf.edu/data/Base/AK_WRF/Historical_Projected_Hourly_Winds_1980_2099/asos/wind_hourly_asos_{}_1980-2015.csv',
        'past_wrf_era_interim': 'http://data.snap.uaf.edu/data/Base/AK_WRF/Historical_Projected_Hourly_Winds_1980_2099/wrf/wind_hourly_wrf_era_interim_{}_1980-2014.csv',
        'future_wrf_gfdl_cm3': 'http://data.snap.uaf.edu/data/Base/AK_WRF/Historical_Projected_Hourly_Winds_1980_2099/wrf/wind_hourly_wrf_gfdl_cm3_{}_2015-2099.csv',
        'future_wrf_ncar_ccsm4': 'http://data.snap.uaf.edu/data/Base/AK_WRF/Historical_Projected_Hourly_Winds_1980_2099/wrf/wind_hourly_wrf_ncar_ccsm4_{}_2015-2099.csv'
            }
    
    # Dictionary to store the data frames
    data_frames = {}

    # Loop over locations and download the CSV data
    for region in regions:
        url = url_format[model].format(region)
        data = download_csv(url)

        # Add the data frame to the dictionary using the location name as the key
        data_frames[region] = data

    # Return the dictionary of data frames
    return data_frames

In [5]:
def extract_may_june_dataframes(dataframes):
    """
    Extracts the May/June data from a dictionary of location dataframes.

    Args:
    dataframes (dict): A dictionary of location dataframes, where the keys are location names and the values are dataframes.

    Returns:
    dict: A dictionary of location dataframes containing only the May/June data.
    """
    output = {}
    for location, df in dataframes.items():
        # Convert the 'ts' column to a datetime object
        df['ts'] = pd.to_datetime(df['ts'])
        
        # Select only the data from May and June
        may_june_data = df.loc[(df['ts'].dt.month == 5) | (df['ts'].dt.month == 6)]
        
        # Store the May/June data in the output dictionary
        output[location] = may_june_data
    
    return output


In [6]:
def extract_jan_to_april_dataframes(dataframes):
    """
    Extracts the May/June data from a dictionary of location dataframes.

    Args:
    dataframes (dict): A dictionary of location dataframes, where the keys are location names and the values are dataframes.

    Returns:
    dict: A dictionary of location dataframes containing only the May/June data.
    """
    output = {}
    for location, df in dataframes.items():
        # Convert the 'ts' column to a datetime object
        df['ts'] = pd.to_datetime(df['ts'])
        
        # Select only the data from May and June
        jan_apr_data = df.loc[(df['ts'].dt.month == 1) | 
                               (df['ts'].dt.month == 2) |
                               (df['ts'].dt.month == 3) |
                               (df['ts'].dt.month == 4)]
        
        # Store the May/June data in the output dictionary
        output[location] = jan_apr_data
    
    return output


In [7]:
# spring_interior = extract_may_june_dataframes(wind_interior)
# pafa = spring_interior['pafa']
# pafa[(pafa['ws']>=10)].groupby(pafa['ts'].dt.year)['ts'].count()

In [8]:
def count_high_wind_by_year(locations_dataframes):
    result = pd.DataFrame()
    
    # Loop through each location dataframe
    for location, df in locations_dataframes.items():
        # Select only the data with wind speed >= 10
        high_wind = df.loc[df['ws'] >= 10]
        
        # Group by year and count the number of rows
        count_by_year = high_wind.groupby(high_wind['ts'].dt.year)['ts'].count()
        
        # Rename the count column to the location name
        count_by_year = count_by_year.rename(location)
        
        # Merge the count column with the result dataframe
        result = pd.concat([result, count_by_year], axis=1, sort=False)
    
    # Reset the index to be the year
    result = result.reset_index()
    
    # Rename the index column to 'year'
    result = result.rename(columns={'index': 'year'})
    
    return result


In [9]:
def spring_wind_by_region(AKregion: str, weatherdf: pd.DataFrame, model: str) -> pd.DataFrame:
    """
    Return a DataFrame with the count of high wind events for each location in a region during May and June
    for each year in the dataset.
    
    Args:
    - AKregion (str): the region for which to compute the wind counts
    - weatherdf (pd.DataFrame): a DataFrame containing weather data for various locations and time periods
    - model (str): one of four SNAP wind models: 
            past_asos, past_wrf_era_interim, future_wrf_gfdl_cm3, future_wrf_ncar_ccsm4
    
    Returns:
    - pd.DataFrame: a DataFrame with the count of high wind events for each location in the region, by year
    
    """
    # group locations by region, convert to lowercase, and get unique codes
    icao_codes_by_region = weatherdf.groupby('region')['icao_code'].apply(lambda x: x.str.lower().unique())
    
    # get list of locations within specified region
    locations = icao_codes_by_region[AKregion]
    
    # download csvs for each location and get data for May and June only
    spring_winds = extract_may_june_dataframes(download_csvs_for_locations(locations, model))
    
    # count the number of high wind events for each location, by year
    location_spring_winds = count_high_wind_by_year(spring_winds)
    
    return location_spring_winds

    

In [10]:
def jan_apr_wind_by_region(AKregion: str, weatherdf: pd.DataFrame, model: str) -> pd.DataFrame:
    """
    Return a DataFrame with the count of high wind events for each location in a region during May and June
    for each year in the dataset.
    
    Args:
    - AKregion (str): the region for which to compute the wind counts
    - weatherdf (pd.DataFrame): a DataFrame containing weather data for various locations and time periods
    - model (str): one of four SNAP wind models: 
            past_asos, past_wrf_era_interim, future_wrf_gfdl_cm3, future_wrf_ncar_ccsm4
    
    Returns:
    - pd.DataFrame: a DataFrame with the count of high wind events for each location in the region, by year
    
    """
    # group locations by region, convert to lowercase, and get unique codes
    icao_codes_by_region = weatherdf.groupby('region')['icao_code'].apply(lambda x: x.str.lower().unique())
    
    # get list of locations within specified region
    locations = icao_codes_by_region[AKregion]
    
    # download csvs for each location and get data for May and June only
    early_winds = extract_jan_to_april_dataframes(download_csvs_for_locations(locations, model))
    
    # count the number of high wind events for each location, by year
    location_early_winds = count_high_wind_by_year(early_winds)
    
    return location_early_winds

    

In [11]:
# spring_interior = extract_may_june_dataframes(wind_IntermontaneBoreal)
# locations_dataframes = {'pafa': spring_interior['pafa'], 'pabi': spring_interior['pabi'], 'pabt': spring_interior['pabt']}
# result = count_high_wind_by_year(locations_dataframes)
# print(result)


In [12]:
#interior_past_asos = spring_wind_by_region("Intermontane Boreal", weather_stations, "past_asos")

In [13]:
#interior_past_asos = interior_past_asos.set_index('year')


In [14]:
#interior_past_asos

In [15]:
#interior_future_ncar = spring_wind_by_region("Intermontane Boreal", weather_stations, "future_wrf_ncar_ccsm4")


In [16]:
#interior_future_ncar['mean'] = interior_future_ncar.mean(axis=1)

In [17]:
#interior_future_ncar[['mean']][50:]

## May and June winds

In [18]:
# # ALASKA RANGE

# ART_past_asos = spring_wind_by_region("Alaska Range Transition", weather_stations, "past_asos").mean(axis=1)
# ART_future_ncar = spring_wind_by_region("Alaska Range Transition", weather_stations, "future_wrf_ncar_ccsm4").mean(axis=1)
# ART_past_interim = spring_wind_by_region("Alaska Range Transition", weather_stations, "past_wrf_era_interim").mean(axis=1)
# ART_future_gfdl_cm3 = spring_wind_by_region("Alaska Range Transition", weather_stations, "future_wrf_gfdl_cm3").mean(axis=1)

In [19]:
# # ALEUTIAN MEADOWS

#AM_past_asos = spring_wind_by_region("Aleutian Meadows", weather_stations, "past_asos").mean(axis=1)
#AM_future_ncar = spring_wind_by_region("Aleutian Meadows", weather_stations, "future_wrf_ncar_ccsm4").mean(axis=1)
#AM_past_interim = spring_wind_by_region("Aleutian Meadows", weather_stations, "past_wrf_era_interim").mean(axis=1)
#AM_future_gfdl_cm3 = spring_wind_by_region("Aleutian Meadows", weather_stations, "future_wrf_gfdl_cm3").mean(axis=1)

In [20]:
# # BERING TAIGA

# BTa_past_asos = spring_wind_by_region("Bering Taiga", weather_stations, "past_asos").mean(axis=1)
# BTa_future_ncar = spring_wind_by_region("Bering Taiga", weather_stations, "future_wrf_ncar_ccsm4").mean(axis=1)
# BTa_past_interim = spring_wind_by_region("Bering Taiga", weather_stations, "past_wrf_era_interim").mean(axis=1)
# BTa_future_gfdl_cm3 = spring_wind_by_region("Bering Taiga", weather_stations, "future_wrf_gfdl_cm3").mean(axis=1)

In [21]:
# # BERING TUNDRA

# BTu_past_asos = spring_wind_by_region("Bering Tundra", weather_stations, "past_asos").mean(axis=1)
# BTu_future_ncar = spring_wind_by_region("Bering Tundra", weather_stations, "future_wrf_ncar_ccsm4").mean(axis=1)
# BTu_past_interim = spring_wind_by_region("Bering Tundra", weather_stations, "past_wrf_era_interim").mean(axis=1)
# BTu_future_gfdl_cm3 = spring_wind_by_region("Bering Tundra", weather_stations, "future_wrf_gfdl_cm3").mean(axis=1)

In [22]:
# #Coastal Rainforests

# CR_past_asos = spring_wind_by_region("Coastal Rainforests", weather_stations, "past_asos").mean(axis=1)
# CR_future_ncar = spring_wind_by_region("Coastal Rainforests", weather_stations, "future_wrf_ncar_ccsm4").mean(axis=1)
# CR_past_interim = spring_wind_by_region("Coastal Rainforests", weather_stations, "past_wrf_era_interim").mean(axis=1)
# CR_future_gfdl_cm3 = spring_wind_by_region("Coastal Rainforests", weather_stations, "future_wrf_gfdl_cm3").mean(axis=1)

In [23]:
# #Copper River & Wrangell Mountains

# CRWM_past_asos = spring_wind_by_region("Copper River & Wrangell Mountains", weather_stations, "past_asos").mean(axis=1)
# CRWM_future_ncar = spring_wind_by_region("Copper River & Wrangell Mountains", weather_stations, "future_wrf_ncar_ccsm4").mean(axis=1)
# CRWM_past_interim = spring_wind_by_region("Copper River & Wrangell Mountains", weather_stations, "past_wrf_era_interim").mean(axis=1)
# CRWM_future_gfdl_cm3 = spring_wind_by_region("Copper River & Wrangell Mountains", weather_stations, "future_wrf_gfdl_cm3").mean(axis=1)

In [24]:
#NORTH SLOPE

#NS_past_asos = spring_wind_by_region("North Slope", weather_stations, "past_asos").mean(axis=1)
#NS_future_ncar = spring_wind_by_region("North Slope", weather_stations, "future_wrf_ncar_ccsm4").mean(axis=1)
#NS_past_interim = spring_wind_by_region("North Slope", weather_stations, "past_wrf_era_interim").mean(axis=1)
#NS_future_gfdl_cm3 = spring_wind_by_region("North Slope", weather_stations, "future_wrf_gfdl_cm3").mean(axis=1)

## January through April winds

In [27]:
# #NORTH SLOPE

#NS_past_asos_early = jan_apr_wind_by_region("North Slope", weather_stations, "past_asos").mean(axis=1)
#NS_future_ncar_early = jan_apr_wind_by_region("North Slope", weather_stations, "future_wrf_ncar_ccsm4").mean(axis=1)
#NS_past_interim_early = jan_apr_wind_by_region("North Slope", weather_stations, "past_wrf_era_interim").mean(axis=1)
#NS_future_gfdl_cm3_early = jan_apr_wind_by_region("North Slope", weather_stations, "future_wrf_gfdl_cm3").mean(axis=1)

In [26]:
NS_past_asos_early

0     1558.666667
1     1352.333333
2     1480.333333
3     1552.333333
4     1482.500000
5     1396.500000
6     1298.500000
7     1216.500000
8     1266.166667
9     1027.833333
10     928.166667
11    1171.400000
12    1181.200000
13    1435.166667
14    1336.000000
15    1337.500000
16    1329.666667
17    1328.000000
18    1257.000000
19    1242.166667
20    1112.333333
21    1057.333333
22    1136.333333
23    1235.000000
24    1427.166667
25    1368.833333
26    1241.333333
27    1314.333333
28    1307.666667
29    1408.500000
30    1334.333333
31    1344.833333
32    1324.333333
33    1367.666667
34    1415.666667
35     421.600000
dtype: float64

In [28]:
NS_past_interim_early

0     1733.833333
1     1403.333333
2     1592.666667
3     1673.500000
4     1407.500000
5     1622.166667
6     1499.666667
7     1511.166667
8     1479.166667
9     1598.333333
10    1432.500000
11    1456.666667
12    1434.666667
13    1556.500000
14    1562.666667
15    1450.833333
16    1466.000000
17    1348.333333
18    1437.666667
19    1407.500000
20    1608.333333
21    1695.166667
22    1503.666667
23    1442.333333
24    1717.666667
25    1655.833333
26    1488.166667
27    1384.833333
28    1489.166667
29    1473.166667
30    1523.333333
31    1418.166667
32    1523.166667
33    1722.833333
34    1445.666667
dtype: float64

In [29]:
NS_future_ncar_early[0:50]

0     1422.333333
1     1645.833333
2     1443.333333
3     1601.333333
4     1792.000000
5     1461.666667
6     1496.500000
7     1597.833333
8     1608.000000
9     1684.500000
10    1683.166667
11    1600.500000
12    1433.500000
13    1602.166667
14    1428.333333
15    1766.833333
16    1403.166667
17    1478.000000
18    1501.666667
19    1667.333333
20    1348.500000
21    1717.333333
22    1420.666667
23    1386.166667
24    1496.500000
25    1303.833333
26    1653.833333
27    1456.833333
28    1694.833333
29    1556.500000
30    1553.000000
31    1571.333333
32    1528.000000
33    1531.500000
34    1561.333333
35    1507.833333
36    1410.000000
37    1712.333333
38    1625.833333
39    1704.333333
40    1619.666667
41    1651.833333
42    1446.500000
43    1500.833333
44    1537.500000
45    1543.500000
46    1583.666667
47    1607.333333
48    1583.166667
49    1508.166667
dtype: float64

In [30]:
NS_future_ncar_early[50:]

50    1561.166667
51    1481.666667
52    1672.000000
53    1661.500000
54    1652.833333
55    1623.833333
56    1644.500000
57    1503.500000
58    1734.833333
59    1871.333333
60    1741.333333
61    1536.833333
62    1573.333333
63    1533.166667
64    1639.833333
65    1786.500000
66    1558.166667
67    1529.166667
68    1647.333333
69    1475.000000
70    1500.500000
71    1630.166667
72    1570.166667
73    1559.333333
74    1726.166667
75    1628.666667
76    1807.500000
77    1705.166667
78    1770.000000
79    1575.666667
80    1781.000000
81    1671.666667
82    1510.333333
83    1846.333333
84    1591.000000
dtype: float64

In [31]:
NS_future_gfdl_cm3_early[0:50]

0     1486.166667
1     1437.000000
2     1388.833333
3     1505.333333
4     1445.500000
5     1389.500000
6     1441.000000
7     1519.666667
8     1560.500000
9     1459.333333
10    1510.000000
11    1591.666667
12    1391.666667
13    1353.666667
14    1485.500000
15    1323.666667
16    1502.000000
17    1588.166667
18    1732.666667
19    1465.500000
20    1562.000000
21    1712.833333
22    1498.000000
23    1476.833333
24    1510.833333
25    1466.166667
26    1444.333333
27    1333.666667
28    1760.833333
29    1708.666667
30    1403.166667
31    1574.500000
32    1695.000000
33    1506.333333
34    1752.833333
35    1620.833333
36    1727.000000
37    1585.000000
38    1596.000000
39    1640.500000
40    1631.333333
41    1813.166667
42    1625.333333
43    1586.333333
44    1517.333333
45    1778.833333
46    1713.833333
47    1667.666667
48    1693.833333
49    1719.166667
dtype: float64

In [32]:
NS_future_gfdl_cm3_early[50:]

50    1680.333333
51    1674.833333
52    1690.166667
53    1693.000000
54    1538.166667
55    1507.000000
56    1687.000000
57    1729.333333
58    1691.000000
59    1805.500000
60    1801.000000
61    1675.833333
62    1631.000000
63    1652.833333
64    1764.333333
65    1781.166667
66    1793.833333
67    1608.500000
68    1812.833333
69    1653.000000
70    1696.166667
71    1653.166667
72    1791.833333
73    1832.166667
74    1609.666667
75    1802.333333
76    1613.166667
77    1640.166667
78    1718.166667
79    1579.833333
80    1558.000000
81    1704.666667
82    1736.833333
83    1719.500000
84    1635.000000
dtype: float64

In [33]:
# # BERING TUNDRA

# BTu_past_asos_early = jan_apr_wind_by_region("Bering Tundra", weather_stations, "past_asos").mean(axis=1)
# BTu_future_ncar_early = jan_apr_wind_by_region("Bering Tundra", weather_stations, "future_wrf_ncar_ccsm4").mean(axis=1)
# BTu_past_interim_early = jan_apr_wind_by_region("Bering Tundra", weather_stations, "past_wrf_era_interim").mean(axis=1)
# BTu_future_gfdl_cm3_early = jan_apr_wind_by_region("Bering Tundra", weather_stations, "future_wrf_gfdl_cm3").mean(axis=1)

In [34]:
BTu_past_asos_early

0     1788.0
1     1813.5
2     1757.5
3     1708.5
4     1446.8
5     1038.8
6      967.2
7     1088.0
8     1095.6
9     1211.2
10     854.2
11    1262.4
12    1257.0
13    1411.0
14    1504.0
15    1348.0
16    1563.6
17    1145.6
18    1380.0
19    1464.6
20    1503.0
21    1510.4
22    1514.0
23    1516.4
24    1754.4
25    1518.6
26    1600.2
27    1551.6
28    1493.0
29    1411.2
30    1429.8
31    1511.2
32    1375.0
33    1576.6
34    1321.6
35     412.0
dtype: float64

In [35]:
BTu_past_interim_early

0     1735.4
1     1695.4
2     1790.6
3     1621.4
4     1640.2
5     1620.4
6     1619.6
7     1722.6
8     1880.6
9     1813.6
10    1407.2
11    1743.8
12    1469.6
13    1585.6
14    1786.6
15    1625.8
16    1727.4
17    1518.8
18    1734.4
19    1603.8
20    1742.0
21    1878.0
22    1752.6
23    1597.8
24    1860.8
25    1855.0
26    1917.8
27    1730.6
28    1824.4
29    1714.8
30    1630.0
31    1619.6
32    1488.8
33    1679.2
34    1755.2
dtype: float64

In [36]:
BTu_future_ncar_early[0:50]

0     1754.4
1     1882.8
2     1734.6
3     1819.4
4     1666.2
5     1708.4
6     1496.2
7     1787.0
8     1725.0
9     1877.8
10    1575.8
11    1607.8
12    1647.4
13    1750.2
14    1622.6
15    1737.0
16    1769.4
17    1613.6
18    1727.0
19    1967.8
20    1656.6
21    1978.6
22    1720.2
23    1635.2
24    1701.8
25    1580.2
26    1742.6
27    1626.8
28    2113.6
29    1772.4
30    1704.4
31    1903.4
32    1863.2
33    1673.0
34    1700.2
35    1555.8
36    1548.4
37    2018.6
38    1683.4
39    1823.0
40    1962.2
41    1818.4
42    1619.6
43    1840.2
44    1921.4
45    1667.0
46    2006.4
47    2030.4
48    1982.8
49    1747.8
dtype: float64

In [37]:
BTu_future_ncar_early[50:]

50    1621.8
51    1747.8
52    1819.6
53    1979.8
54    1935.6
55    2072.6
56    2188.4
57    1697.4
58    1872.6
59    1928.0
60    1915.2
61    1762.6
62    1808.6
63    1865.8
64    1986.6
65    2033.6
66    1868.2
67    1546.6
68    1701.2
69    1955.6
70    1874.8
71    1701.8
72    1951.0
73    2143.2
74    2030.4
75    1829.0
76    1936.4
77    2115.0
78    1797.6
79    2030.0
80    1976.0
81    1941.0
82    1794.4
83    2056.0
84    1807.8
dtype: float64

In [38]:
BTu_future_gfdl_cm3_early[0:50]

0     1702.0
1     1731.4
2     1828.4
3     1733.8
4     1938.2
5     1611.2
6     1539.8
7     1842.0
8     1784.0
9     1706.6
10    1587.8
11    1811.6
12    1766.0
13    1540.4
14    1811.4
15    1670.8
16    1760.0
17    1792.2
18    1905.8
19    1793.6
20    1693.0
21    1853.8
22    1846.2
23    1807.2
24    1839.6
25    1698.2
26    1607.4
27    1725.0
28    2014.2
29    1817.0
30    1888.8
31    1861.4
32    1919.6
33    1773.0
34    1880.4
35    1904.2
36    1959.4
37    2002.4
38    1812.0
39    1820.4
40    1849.8
41    1795.8
42    1918.6
43    1821.6
44    1842.0
45    1869.6
46    1831.0
47    1984.0
48    1924.0
49    1759.2
dtype: float64

In [39]:
BTu_future_gfdl_cm3_early[50:]

50    1702.8
51    1831.0
52    1933.6
53    1986.0
54    1925.6
55    1849.0
56    1916.0
57    1933.4
58    1996.2
59    2168.8
60    1935.8
61    1861.4
62    1922.0
63    1984.6
64    2010.4
65    1957.0
66    1931.2
67    1868.0
68    1958.0
69    1881.4
70    2025.6
71    1913.4
72    2057.6
73    1950.4
74    2038.4
75    2041.4
76    1957.0
77    1720.8
78    2024.4
79    1919.4
80    1946.8
81    1770.2
82    2111.4
83    2041.4
84    2148.6
dtype: float64

In [40]:
# # BERING TAIGA

# BTa_past_asos_early = jan_apr_wind_by_region("Bering Taiga", weather_stations, "past_asos").mean(axis=1)
# BTa_future_ncar_early = jan_apr_wind_by_region("Bering Taiga", weather_stations, "future_wrf_ncar_ccsm4").mean(axis=1)
# BTa_past_interim_early = jan_apr_wind_by_region("Bering Taiga", weather_stations, "past_wrf_era_interim").mean(axis=1)
# BTa_future_gfdl_cm3_early = jan_apr_wind_by_region("Bering Taiga", weather_stations, "future_wrf_gfdl_cm3").mean(axis=1)

In [41]:
BTa_past_asos_early

0     1488.000000
1     1765.285714
2     1762.857143
3     1701.000000
4     1555.000000
5     1275.625000
6     1222.250000
7     1205.125000
8     1453.125000
9     1219.750000
10    1110.625000
11    1155.500000
12    1277.000000
13    1662.375000
14    1585.125000
15    1495.250000
16    1545.625000
17    1380.750000
18    1653.500000
19    1474.000000
20    1243.125000
21    1364.375000
22    1624.125000
23    1735.125000
24    1587.000000
25    1633.375000
26    1590.125000
27    1482.500000
28    1527.750000
29    1541.250000
30    1522.250000
31    1579.250000
32    1440.875000
33    1624.875000
34    1676.375000
35     511.000000
dtype: float64

In [42]:
BTa_past_interim_early

0     1821.500
1     1797.250
2     1828.375
3     1824.875
4     1763.750
5     1804.500
6     1662.000
7     1769.625
8     1965.000
9     1656.750
10    1530.250
11    1756.250
12    1584.250
13    1756.750
14    1765.125
15    1687.250
16    1842.500
17    1703.625
18    1899.000
19    1674.250
20    1728.625
21    1822.500
22    1818.875
23    1875.625
24    1760.125
25    1745.375
26    1763.625
27    1646.875
28    1656.750
29    1775.625
30    1693.625
31    1698.375
32    1615.125
33    1713.000
34    1882.125
dtype: float64

In [43]:
BTa_future_ncar_early[0:50]

0     1934.250
1     1848.750
2     1831.250
3     1762.125
4     1782.375
5     1767.500
6     1684.125
7     1854.500
8     1789.125
9     1985.125
10    1837.000
11    1705.375
12    1746.000
13    1847.875
14    1692.250
15    1808.625
16    1821.250
17    1807.875
18    1854.375
19    1879.375
20    1769.625
21    1817.625
22    1632.750
23    1766.000
24    1813.375
25    1616.250
26    1691.000
27    1776.250
28    2040.625
29    1769.875
30    1849.500
31    1884.000
32    1717.250
33    1803.750
34    1696.000
35    1843.500
36    1594.000
37    1905.125
38    1757.625
39    1851.375
40    1971.375
41    1900.000
42    1663.000
43    1809.250
44    1825.500
45    1760.875
46    1929.000
47    1929.500
48    1884.250
49    1817.375
dtype: float64

In [44]:
BTa_future_ncar_early[50:]

50    1741.750
51    1760.375
52    1869.875
53    2070.875
54    1915.500
55    1923.125
56    1936.000
57    1806.375
58    1948.250
59    1882.125
60    2009.375
61    1755.250
62    1889.750
63    1782.000
64    1954.750
65    1947.375
66    1747.250
67    1577.000
68    1776.125
69    1891.375
70    1783.125
71    1763.250
72    1816.125
73    1958.625
74    1958.625
75    1830.875
76    1742.375
77    2052.500
78    1732.000
79    1947.875
80    1936.750
81    1921.625
82    1816.000
83    2008.875
84    1739.500
dtype: float64

In [45]:
BTa_future_gfdl_cm3_early[0:50]

0     1868.625
1     1754.875
2     1920.500
3     1711.500
4     1928.375
5     1570.500
6     1707.500
7     1929.750
8     1783.875
9     1851.750
10    1854.375
11    1817.500
12    1864.750
13    1520.500
14    1751.500
15    1633.750
16    1800.625
17    1770.250
18    1914.625
19    1839.875
20    1765.875
21    1812.750
22    1835.000
23    1654.875
24    1944.375
25    1566.625
26    1648.625
27    1677.500
28    1902.625
29    1725.375
30    1883.250
31    1822.750
32    1929.500
33    1569.750
34    1715.875
35    1889.750
36    1855.625
37    2051.375
38    1642.000
39    1873.250
40    1857.000
41    1787.375
42    1903.625
43    1729.500
44    1874.250
45    1792.000
46    1817.375
47    1805.125
48    1829.375
49    1730.375
dtype: float64

In [46]:
BTa_future_gfdl_cm3_early[50:]

50    1922.000
51    1699.000
52    1846.250
53    1844.125
54    1906.875
55    1879.500
56    1808.000
57    2018.625
58    1764.250
59    1990.375
60    1843.250
61    1779.500
62    1828.000
63    1838.125
64    1776.125
65    1758.375
66    1713.125
67    1779.500
68    1762.750
69    1945.875
70    2063.250
71    1839.625
72    1861.000
73    1938.250
74    1965.625
75    1892.625
76    1886.500
77    1698.125
78    1822.375
79    1796.250
80    1838.750
81    1652.125
82    1862.625
83    2003.125
84    2095.375
dtype: float64

In [47]:
# INTERMONTANE BOREAL

interior_past_asos_early = jan_apr_wind_by_region("Intermontane Boreal", weather_stations, "past_asos").mean(axis=1)
interior_future_ncar_early = jan_apr_wind_by_region("Intermontane Boreal", weather_stations, "future_wrf_ncar_ccsm4").mean(axis=1)
interior_past_interim_early = jan_apr_wind_by_region("Intermontane Boreal", weather_stations, "past_wrf_era_interim").mean(axis=1)
interior_future_gfdl_cm3_early = jan_apr_wind_by_region("Intermontane Boreal", weather_stations, "future_wrf_gfdl_cm3").mean(axis=1)


In [48]:
interior_past_asos_early

0     528.058824
1     491.000000
2     479.411765
3     477.470588
4     363.411765
5     381.000000
6     361.941176
7     396.823529
8     540.058824
9     400.058824
10    331.875000
11    361.588235
12    340.823529
13    472.312500
14    516.470588
15    428.529412
16    462.941176
17    395.058824
18    457.823529
19    464.000000
20    459.875000
21    516.117647
22    499.705882
23    501.588235
24    490.235294
25    456.882353
26    466.058824
27    382.117647
28    431.647059
29    533.058824
30    415.117647
31    531.352941
32    448.705882
33    543.882353
34    520.764706
35    404.800000
dtype: float64

In [49]:
interior_past_interim_early

0     634.588235
1     557.647059
2     558.882353
3     584.411765
4     512.058824
5     479.117647
6     479.176471
7     499.882353
8     638.529412
9     462.470588
10    387.294118
11    541.176471
12    486.058824
13    470.352941
14    609.058824
15    425.529412
16    490.235294
17    473.941176
18    484.000000
19    505.176471
20    553.411765
21    574.764706
22    483.235294
23    427.058824
24    579.647059
25    506.000000
26    512.470588
27    471.294118
28    576.470588
29    593.411765
30    528.705882
31    488.588235
32    451.411765
33    594.941176
34    502.823529
dtype: float64

In [50]:
interior_future_ncar_early[0:50]

0     509.529412
1     494.764706
2     469.823529
3     737.529412
4     610.294118
5     434.588235
6     447.411765
7     586.823529
8     536.764706
9     591.941176
10    535.705882
11    492.764706
12    442.000000
13    586.588235
14    443.000000
15    583.588235
16    542.000000
17    457.882353
18    530.176471
19    617.235294
20    540.647059
21    638.705882
22    552.352941
23    451.823529
24    565.823529
25    418.941176
26    535.764706
27    432.058824
28    711.411765
29    558.411765
30    639.882353
31    618.058824
32    642.294118
33    534.411765
34    440.411765
35    472.235294
36    429.588235
37    864.352941
38    437.411765
39    668.705882
40    654.941176
41    562.882353
42    466.764706
43    593.470588
44    598.882353
45    585.823529
46    503.882353
47    697.647059
48    666.058824
49    515.000000
dtype: float64

In [51]:
interior_future_ncar_early[50:]

50    603.705882
51    546.647059
52    582.823529
53    663.117647
54    687.000000
55    600.882353
56    847.882353
57    467.000000
58    615.294118
59    686.647059
60    718.176471
61    448.235294
62    581.352941
63    558.823529
64    529.882353
65    622.588235
66    571.117647
67    400.705882
68    432.294118
69    647.235294
70    506.705882
71    553.529412
72    560.352941
73    615.294118
74    642.470588
75    661.470588
76    578.117647
77    643.000000
78    537.411765
79    559.647059
80    779.352941
81    584.000000
82    483.235294
83    766.176471
84    442.352941
dtype: float64

In [52]:
interior_future_gfdl_cm3_early[0:50]

0     539.117647
1     528.058824
2     430.529412
3     486.294118
4     510.000000
5     440.647059
6     463.882353
7     638.176471
8     606.470588
9     526.470588
10    579.941176
11    551.352941
12    479.058824
13    522.235294
14    563.470588
15    427.294118
16    563.411765
17    590.529412
18    575.294118
19    492.588235
20    525.058824
21    608.411765
22    524.705882
23    529.529412
24    497.941176
25    501.705882
26    390.588235
27    516.823529
28    581.411765
29    620.058824
30    567.176471
31    550.764706
32    592.294118
33    425.941176
34    564.058824
35    579.000000
36    552.470588
37    532.117647
38    534.823529
39    503.823529
40    564.235294
41    554.294118
42    559.058824
43    451.529412
44    603.882353
45    501.823529
46    670.647059
47    615.352941
48    599.588235
49    461.176471
dtype: float64

In [53]:
interior_future_gfdl_cm3_early[50:]

50    580.352941
51    552.352941
52    623.647059
53    624.058824
54    578.941176
55    427.823529
56    638.235294
57    614.058824
58    701.882353
59    696.588235
60    642.352941
61    629.588235
62    478.235294
63    610.529412
64    604.705882
65    570.529412
66    526.411765
67    544.588235
68    664.705882
69    499.411765
70    614.941176
71    550.411765
72    749.117647
73    617.705882
74    621.294118
75    682.000000
76    577.470588
77    500.235294
78    645.529412
79    642.764706
80    587.705882
81    586.529412
82    651.647059
83    728.117647
84    731.764706
dtype: float64